In [ ]:
import pandas as pd
import numpy as np
from FP import get_df_FP, get_df_extendida_FP
from CD import get_df_CD, get_df_extendida_CD
from FURANOS import get_df_FUR, get_df_extendida_FUR
from ECC import get_df_ECC, get_df_extendida_ECC
from functools import reduce

# =============================
# PESOS DE LOS ÍNDICES
# =============================
pesos_AIS = {
    'V': 1,
    'ECC': 5,
    'FUR': 2,
    'FPDEVANADO':2,
    'CD':3
}
# =============================
# FUNCIÓN PARA CALCULAR AIS
# =============================
def calcular_AIS(row, pesos):
    valores = [row[col] * peso for col, peso in pesos.items() if col in row and pd.notna(row[col])]
    total_peso = sum(peso for col, peso in pesos.items() if col in row and pd.notna(row[col]))
    return sum(valores) / total_peso if total_peso > 0 else float("nan")

In [ ]:
# =============================
# FUNCIÓN PRINCIPAL
# =============================
def get_df_detalles_AIS():
    FPDEVANADO = get_df_FP()
    CD = get_df_CD()
    FUR = get_df_FUR()
    ECC = get_df_ECC()
    tablas = {
        "FPDEVANADO": FPDEVANADO,
        "CD": CD,
        "FUR": FUR,
        "ECC": ECC
    }
    # Filtramos los que no sean None
    tablas = {k: v for k, v in tablas.items() if v is not None}

    if not tablas:
        return pd.DataFrame()
    
    # Merge progresivo de todas las tablas (asumimos que todas ya tienen SERIE + FECHA extendidos)
    resultado = reduce(
        lambda left, right: pd.merge(left, right, on=["SERIE", "FECHA"], how="outer"),
        tablas.values()
    )
    # Asegurar tipo datetime en FECHA
    resultado["FECHA"] = pd.to_datetime(resultado["FECHA"], errors="coerce")

    # Ordenamos por SERIE y FECHA DE MUESTRA
    resultado = resultado.sort_values(by=["SERIE", "FECHA"]).reset_index(drop=True)

    # ============================
    # Calcular AIS
    # ============================
    resultado["AIS"] = resultado.apply(lambda row: calcular_AIS(row, pesos_AIS), axis=1)

    # Selección de columnas finales
    columnas_finales = ["SERIE", "FECHA", "AIS"] + [col for col in tablas.keys()]
    resultado = resultado[columnas_finales]
    return resultado

,SERIE,FECHA,FPDEVANADO
0,760026-01,2025-04-06,3.0
1,760026-01,2024-12-15,5.0
2,760026-01,2024-11-13,5.0
3,760026-01,2024-11-12,5.0
4,760026-01,2024-09-08,5.0
5,760026-01,2020-12-06,3.0
6,760026-01,2013-06-08,3.0
7,760026-01,2006-12-06,3.0
8,146660T3,2024-08-27,1.0
9,146660T3,2019-10-06,1.0


In [28]:
# =============================
# PESOS DE LOS ÍNDICES
# =============================
pesos_AIS = {
    'V': 1,
    'ICC': 5,
    'FUR': 2,
    'FPDEVANADO':2,
    'CD':3
}
# =============================
# FUNCIÓN PARA CALCULAR AIS
# =============================
def calcular_AIS(row, pesos):
    valores = [row[col] * peso for col, peso in pesos.items() if col in row and pd.notna(row[col])]
    total_peso = sum(peso for col, peso in pesos.items() if col in row and pd.notna(row[col]))
    return sum(valores) / total_peso if total_peso > 0 else float("nan")

# =============================
# FUNCIÓN PRINCIPAL
# =============================
def get_df_detalles_AIS():
    FPDEVANADO = get_df_FP()
    CD = get_df_CD()
    tablas = {
        "FPDEVANADO": FPDEVANADO,
        "CD": CD,
        # "RDIS": RDIS,
    }
    # Filtramos los que no sean None
    tablas = {k: v for k, v in tablas.items() if v is not None}

    if not tablas:
        return pd.DataFrame()
    
    # Merge progresivo de todas las tablas (asumimos que todas ya tienen SERIE + FECHA extendidos)
    resultado = reduce(
        lambda left, right: pd.merge(left, right, on=["SERIE", "FECHA"], how="outer"),
        tablas.values()
    )
    # Asegurar tipo datetime en FECHA
    resultado["FECHA"] = pd.to_datetime(resultado["FECHA"], errors="coerce")

    # Ordenamos por SERIE y FECHA DE MUESTRA
    resultado = resultado.sort_values(by=["SERIE", "FECHA"]).reset_index(drop=True)

    # ============================
    # Calcular AIS
    # ============================
    resultado["AIS"] = resultado.apply(lambda row: calcular_AIS(row, pesos_AIS), axis=1)

    # Selección de columnas finales
    columnas_finales = ["SERIE", "FECHA", "AIS"] + [col for col in tablas.keys()]
    resultado = resultado[columnas_finales]
    return resultado
df_arr = get_df_detalles_AIS()
df_arr_detalles = df_arr[['SERIE','FECHA','AIS']]
df_arr.head()

,SERIE,FECHA,AIS,FPDEVANADO,CD
0,146660T3,2014-03-22,1.0,1.0,1.0
1,146660T3,2014-04-20,1.0,1.0,1.0
2,146660T3,2015-05-28,1.8,3.0,1.0
3,146660T3,2019-10-06,1.0,1.0,1.0
4,146660T3,2024-08-27,2.2,1.0,3.0


In [29]:
df_arr_detalles.head()

,SERIE,FECHA,AIS
0,146660T3,2014-03-22,1.0
1,146660T3,2014-04-20,1.0
2,146660T3,2015-05-28,1.8
3,146660T3,2019-10-06,1.0
4,146660T3,2024-08-27,2.2


In [ ]:
def get_df_detalles_ext_AIS():
    FPDEVANADO = get_df_extendida_FP()
    CD = get_df_extendida_CD()
    tablas = {
        "FPDEVANADO": FPDEVANADO,
        "CD": CD,
        # "RDIS": RDIS,
    }
    # Filtramos los que no sean None
    tablas = {k: v for k, v in tablas.items() if v is not None}

    if not tablas:
        return pd.DataFrame()
    
    # Merge progresivo de todas las tablas (asumimos que todas ya tienen SERIE + FECHA extendidos)
    resultado = reduce(
        lambda left, right: pd.merge(left, right, on=["SERIE", "FECHA"], how="outer"),
        tablas.values()
    )
    # Asegurar tipo datetime en FECHA
    resultado["FECHA"] = pd.to_datetime(resultado["FECHA"], errors="coerce")

    # Ordenamos por SERIE y FECHA DE MUESTRA
    resultado = resultado.sort_values(by=["SERIE", "FECHA"]).reset_index(drop=True)

    # ============================
    # Calcular AIS
    # ============================
    resultado["AIS"] = resultado.apply(lambda row: calcular_AIS(row, pesos_AIS), axis=1)

    # Selección de columnas finales
    columnas_finales = ["SERIE", "FECHA", "AIS"] + [col for col in tablas.keys()]
    resultado = resultado[columnas_finales]
    return resultado

In [ ]:
df_AIS_detalles = get_df_detalles_ext_AIS()
df_AIS = df_AIS_detalles[['SERIE','FECHA','AIS']]


def get_df_extendida_AIS():
    return df_AIS

df_arr = get_df_detalles_AIS()
df_arr_detalles = df_arr[['SERIE','FECHA','AIS']]
print(df_arr.head())
print(get_df_detalles_ext_AIS().head())
print(df_AIS)

,SERIE,FECHA,AIS,FPDEVANADO,CD
0,146660T3,2025-01-01,2.2,1.0,3.0
1,146660T3,2025-01-02,2.2,1.0,3.0
2,146660T3,2025-01-03,2.2,1.0,3.0
3,146660T3,2025-01-04,2.2,1.0,3.0
4,146660T3,2025-01-05,2.2,1.0,3.0
...,...,...,...,...,...
539,760026-01,2025-09-25,3.0,3.0,3.0
540,760026-01,2025-09-26,3.0,3.0,3.0
541,760026-01,2025-09-27,3.0,3.0,3.0
542,760026-01,2025-09-28,3.0,3.0,3.0
